<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%ED%83%80%EC%9E%84%EB%9E%A9%EC%8A%A4_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine을 활용한 타임랩스 생성

In [6]:
!pip install -q -U ffmpeg-python
!pip install -q -U geokakao

In [31]:
import ee
import geemap
import geokakao as gk
import pandas as pd
import math

from ipyleaflet import TileLayer

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [9]:
data = {'Name': ['태화강국가정원'],
        'Address': ['울산 중구 태화동 107']}
df = pd.DataFrame(data)
df.head()

,Name,Address
0,태화강국가정원,울산 중구 태화동 107


In [10]:
gk.add_coordinates_to_dataframe(df, 'Address')
df.head()

,Name,Address,decimalLatitude,decimalLongitude
0,태화강국가정원,울산 중구 태화동 107,35.5500785012707,129.299077253026


In [26]:
df.decimalLatitude.values

array(['35.5500785012707'], dtype=object)

In [48]:
# Vworld 영상지도 객체
vworld_satellite = TileLayer(
    url='https://xdworld.vworld.kr/2d/Satellite/service/{z}/{x}/{y}.jpeg',
    name='Vworld Satellite',
    attribution='Vworld'
)
# Vworld 하이브리드지도 객체
vworld_hybrid = TileLayer(
    url='https://xdworld.vworld.kr/2d/Hybrid/service/{z}/{x}/{y}.png',
    name='Vworld Hybrid',
    attribution='Vworld',
)

# 태화강국가정원 위치 가시화
m = geemap.Map(width="800px", height="500px")
m.add_layer(vworld_satellite)
m.add_layer(vworld_hybrid)

latitude, longitude = df.loc[0, ['decimalLatitude', 'decimalLongitude']].astype(float)
location = ee.Geometry.Point([longitude, latitude])
m.centerObject(location, 16) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[35.550078501270704, 129.299077253026], controls=(WidgetControl(options=['position', 'transparent_b…

In [55]:
def calculate_bbox(longitude, latitude, width, height):
    # 지구 반경 (미터)
    earth_radius = 6378137
    # 1도당 미터 (위도)
    meter_per_degree_lat = 2 * math.pi * earth_radius / 360
    # 현재 위도에서 1도당 미터 (경도)
    meter_per_degree_lng = meter_per_degree_lat * math.cos(math.radians(latitude))

    # 폭과 높이를 각각 경도와 위도의 차이로 변환
    delta_lng = width / 2 / meter_per_degree_lng
    delta_lat = height / 2 / meter_per_degree_lat

    # BBox의 경계 좌표 계산
    west = longitude - delta_lng
    east = longitude + delta_lng
    south = latitude - delta_lat
    north = latitude + delta_lat

    # ee.Geometry.BBox 객체 생성 및 반환
    bbox = ee.Geometry.BBox(west, south, east, north)
    return bbox

# 중심점 좌표 및 상자 크기 지정
width = 4000
height = 2000

# BBox 생성
roi = calculate_bbox(longitude, latitude, width, height)

In [57]:
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif="sentinel1.gif",
    start_year=2022,
    end_year=2022,
    start_date="04-01",
    end_date="08-01",
    frequency="day",
    vis_params={"min": -30, "max": 0},
    palette="Greys",
    frames_per_second=3,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel1.gif


Output()